In [1]:
# code/python

class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next
        
    def __repr__(self):
        return f"ListNode({self.val})"

In [14]:
def build_list(values, cycle_pos=-1):
    """
    Build a linked list from values list.
    If cycle_pos != -1, create a cycle pointing last node to node at index cycle_pos.
    Returns head.
    """
    if not values:
        return None
    nodes = [ListNode(v) for v in values]
    for i in range(len(nodes)-1):
        nodes[i].next = nodes[i+1]
    if cycle_pos != -1 and 0 <= cycle_pos < len(nodes):
        nodes[-1].next = nodes[cycle_pos]
    return nodes[0]

def limited_print(head, limit=30):
    """Print list values up to limit nodes to avoid infinite loops."""
    out = []
    cur = head
    steps = 0
    while cur and steps < limit:
        out.append(str(cur.val))
        cur = cur.next
        steps += 1
    if cur:
        out.append("...")
    return "->".join(out)

def detect_cycle_start(head):
    """
    If cycle exists, return the node where the cycle begins; otherwise return None.
    """
    slow = fast = head
    # detect meeting
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next
        if slow is fast:
            break
    else:
        return None  # no cycle

    # find cycle start
    ptr = head
    while ptr is not slow:
        ptr = ptr.next
        slow = slow.next
    return ptr

def find_middle(head):
    """
    Return middle node. If even length, returns second middle (LeetCode convention).
    """
    slow = fast = head
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next
    return slow

def reverse_list(head):
    prev = None
    cur = head
    while cur:
        nxt = cur.next
        cur.next = prev
        prev = cur
        cur = nxt
    return prev

def is_palindrome(head, restore=True):
    """
    Check if the list is a palindrome. Optionally restore list to original shape.
    """
    if not head or not head.next:
        return True

    # 1. find middle (slow will be start of second half for even/odd)
    slow = fast = head
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next

    # For odd nodes, slow is the exact middle. For palindrome check, start reverse from slow.
    second_head = reverse_list(slow)

    # 3. compare
    p1, p2 = head, second_head
    is_pal = True
    while p2:  # second half shorter or equal
        if p1.val != p2.val:
            is_pal = False
            break
        p1 = p1.next
        p2 = p2.next

    # 4. optional: restore second half
    if restore:
        reverse_list(second_head)

    return is_pal

In [7]:
# Example usage helpers:
def example_cycle_demo():
    head = build_list(['A','B','C','D','E'], cycle_pos=2)
    start = detect_cycle_start(head)
    return head, start

In [8]:
example_cycle_demo()

(ListNode(A), ListNode(C))

In [9]:
def example_middle_demo(vals):
    head = build_list(vals)
    mid = find_middle(head)
    return head, mid

In [11]:
vals = [1,2,4,5,3,6,7,8,10,9]
example_middle_demo(vals)

(ListNode(1), ListNode(6))

In [12]:
def example_palindrome_demo(vals):
    head = build_list(vals)
    pal = is_palindrome(head, restore=True)
    return head, pal

In [13]:
example_palindrome_demo(vals)

(ListNode(1), False)

In [15]:
head = build_list(vals)
t = reverse_list(head)

ListNode(6)

# Day 1 

## Step 1 --> fast–slow pointer problems

##### Perfect 👍 — let’s do this the **“regular” linked list way**,
i.e., we’ll explicitly create a `Node` class, manually link nodes using `next`, and then run **3 fast–slow pointer problems** step-by-step.

We’ll cover:
1️⃣ Detect cycle
2️⃣ Find middle node
3️⃣ Check palindrome

Each with 👉 problem → logic → step-by-step trace → Python code → sample input & output.

---

### 🧩 1. Detect Cycle in a Linked List

#### 🔹 Problem

Given a linked list, check if it has a cycle (loop).
If a cycle exists, return the **node where the cycle begins**, else return `None`.

---

#### 🔹 Logic

Use **Floyd’s Cycle Detection** (Tortoise and Hare):

1. Initialize two pointers:

   * `slow = head`
   * `fast = head`
2. Move:

   * `slow = slow.next`
   * `fast = fast.next.next`
3. If they ever meet → there’s a cycle.
4. To find the cycle start:

   * Keep `slow` at meeting point.
   * Move another pointer `ptr = head`.
   * Move both `slow` and `ptr` one step at a time.
   * They meet at cycle start.

---

#### 🔹 Python Code

```python
class Node:
    def __init__(self, val):
        self.val = val
        self.next = None

# ----------- Detect Cycle Function -----------
def detect_cycle(head):
    slow = fast = head

    # Phase 1: Detect if cycle exists
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next
        if slow == fast:
            break
    else:
        return None  # No cycle

    # Phase 2: Find cycle start
    ptr = head
    while ptr != slow:
        ptr = ptr.next
        slow = slow.next
    return ptr


# ----------- Example Input -----------
A = Node("A")
B = Node("B")
C = Node("C")
D = Node("D")
E = Node("E")

A.next = B
B.next = C
C.next = D
D.next = E
E.next = C  # cycle starts at C

head = A
cycle_start = detect_cycle(head)
print("Cycle starts at:", cycle_start.val if cycle_start else None)
```

---

#### 🔹 Step-by-Step Trace

| Step | `slow`        | `fast`                     | Comment                    |
| ---- | ------------- | -------------------------- | -------------------------- |
| 1    | A             | A                          | start                      |
| 2    | B             | C                          | move slow 1, fast 2        |
| 3    | C             | E                          | move slow 1, fast 2        |
| 4    | D             | D                          | they meet → cycle detected |
| 5    | ptr=A, slow=D | Move both by 1 until meet  |                            |
| 6    | ptr=B, slow=E |                            |                            |
| 7    | ptr=C, slow=C | meet → cycle starts at C ✅ |                            |

✅ **Output:** `Cycle starts at: C`

---

#### 🧩 2. Find Middle of Linked List

#### 🔹 Problem

Given the head of a linked list, find its **middle node**.
If the length is even → return the **second middle node**.

---

#### 🔹 Logic

Use fast and slow pointers:

1. `slow = head`, `fast = head`
2. Move:

   * `slow = slow.next`
   * `fast = fast.next.next`
3. When `fast` or `fast.next` is `None` → `slow` is at middle.

---

#### 🔹 Python Code

```python
def find_middle(head):
    slow = fast = head
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next
    return slow


# Example 1: Odd length
head1 = Node(1)
head1.next = Node(2)
head1.next.next = Node(3)
head1.next.next.next = Node(4)
head1.next.next.next.next = Node(5)

mid = find_middle(head1)
print("Middle (odd):", mid.val)

# Example 2: Even length
head2 = Node(1)
head2.next = Node(2)
head2.next.next = Node(3)
head2.next.next.next = Node(4)

mid2 = find_middle(head2)
print("Middle (even):", mid2.val)
```

---

#### 🔹 Step-by-Step Trace

##### Example 1: 1 → 2 → 3 → 4 → 5

| Step                               | `slow` | `fast` |
| ---------------------------------- | ------ | ------ |
| 1                                  | 1      | 1      |
| 2                                  | 2      | 3      |
| 3                                  | 3      | 5      |
| → `fast` reaches end, `slow` = 3 ✅ |        |        |

✅ **Output:** `Middle (odd): 3`

##### Example 2: 1 → 2 → 3 → 4

| Step                           | `slow` | `fast` |
| ------------------------------ | ------ | ------ |
| 1                              | 1      | 1      |
| 2                              | 2      | 3      |
| 3                              | 3      | None   |
| → `slow` = 3 (second middle) ✅ |        |        |

✅ **Output:** `Middle (even): 3`

---

### 🧩 3. Palindrome Linked List

#### 🔹 Problem

Check if a linked list reads the same forwards and backwards.

---

#### 🔹 Logic

1. Find the middle (slow & fast pointers).
2. Reverse the **second half** of the list.
3. Compare both halves node by node.
4. Restore list (optional).

---

#### 🔹 Python Code

```python
def reverse_list(head):
    prev = None
    current = head
    while current:
        nxt = current.next
        current.next = prev
        prev = current
        current = nxt
    return prev

def is_palindrome(head):
    if not head or not head.next:
        return True

    # Step 1: find middle
    slow = fast = head
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next

    # Step 2: reverse second half
    second_half = reverse_list(slow)
    first_half = head

    # Step 3: compare both halves
    p1, p2 = first_half, second_half
    palindrome = True
    while p2:
        if p1.val != p2.val:
            palindrome = False
            break
        p1 = p1.next
        p2 = p2.next

    # (Optional) restore second half
    reverse_list(second_half)

    return palindrome


# Example
head = Node(1)
head.next = Node(2)
head.next.next = Node(3)
head.next.next.next = Node(2)
head.next.next.next.next = Node(1)

print("Is palindrome:", is_palindrome(head))
```

---

#### 🔹 Step-by-Step Trace

Linked list: **1 → 2 → 3 → 2 → 1**

| Step | Operation               | Result                   |
| ---- | ----------------------- | ------------------------ |
| 1    | Find middle             | slow at node 3           |
| 2    | Reverse second half     | becomes 1 → 2            |
| 3    | Compare halves          | (1,1), (2,2) → all equal |
| 4    | Restore list (optional) | original list restored   |

✅ **Output:** `Is palindrome: True`

---

### ✅ Final Outputs Recap

| Problem      | Input         | Output             |
| ------------ | ------------- | ------------------ |
| Detect Cycle | A→B→C→D→E→(C) | Cycle starts at: C |
| Middle Node  | 1→2→3→4→5     | Middle: 3          |
| Palindrome   | 1→2→3→2→1     | True               |

---

Would you like me to make this code **interactive** (you enter values and it builds + tests all three cases automatically)?
I can write a small menu-driven Python script for that next.

In [28]:
class Node:
    def __init__(self, val):
        self.val = val
        self.next = None

# ----------- Detect Cycle Function -----------
def detect_cycle(head):
    slow = fast = head

    # Phase 1: Detect if cycle exists
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next
        if slow == fast:
            break
    else:
        return None  # No cycle

    # Phase 2: Find cycle start
    ptr = head
    while ptr != slow:
        ptr = ptr.next
        slow = slow.next
    return ptr


# ----------- Example Input -----------
A = Node("A")
B = Node("B")
C = Node("C")
D = Node("D")
E = Node("E")

A.next = B
B.next = C
C.next = D
D.next = E
E.next = C  # cycle starts at C

head = A
cycle_start = detect_cycle(head)
print("Cycle starts at:", cycle_start.val if cycle_start else None)

Cycle starts at: C


In [29]:
def find_middle(head):
    slow = fast = head
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next
    return slow


# Example 1: Odd length
head1 = Node(1)
head1.next = Node(2)
head1.next.next = Node(3)
head1.next.next.next = Node(4)
head1.next.next.next.next = Node(5)

mid = find_middle(head1)
print("Middle (odd):", mid.val)

# Example 2: Even length
head2 = Node(1)
head2.next = Node(2)
head2.next.next = Node(3)
head2.next.next.next = Node(4)

mid2 = find_middle(head2)
print("Middle (even):", mid2.val)

Middle (odd): 3
Middle (even): 3


In [30]:
def reverse_list(head):
    prev = None
    current = head
    while current:
        nxt = current.next
        current.next = prev
        prev = current
        current = nxt
    return prev

def is_palindrome(head):
    if not head or not head.next:
        return True

    # Step 1: find middle
    slow = fast = head
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next

    # Step 2: reverse second half
    second_half = reverse_list(slow)
    first_half = head

    # Step 3: compare both halves
    p1, p2 = first_half, second_half
    palindrome = True
    while p2:
        if p1.val != p2.val:
            palindrome = False
            break
        p1 = p1.next
        p2 = p2.next

    # (Optional) restore second half
    reverse_list(second_half)

    return palindrome


# Example
head = Node(1)
head.next = Node(2)
head.next.next = Node(3)
head.next.next.next = Node(2)
head.next.next.next.next = Node(1)

print("Is palindrome:", is_palindrome(head))

Is palindrome: True


## Step 2--> Sliding Window

Excellent 👏 — you’re moving to one of the **most important DSA patterns: Sliding Window**.

Let’s break it down **from scratch**, focusing on **substrings**, **subarrays**, and **range (longest/shortest)** type problems.

---

### 🧩 Sliding Window Pattern

#### 💡 Concept

The **Sliding Window** technique is used to process **contiguous elements** in a sequence (array or string) efficiently — usually in **O(n)** time.

You maintain a **window** (a range `[left, right]`) that slides over the input to maintain a condition (like sum, distinct characters, etc.).

---

### ⚙️ Two Main Window Types

| Type                | Description                                 | Example                                                     |
| ------------------- | ------------------------------------------- | ----------------------------------------------------------- |
| **Fixed Window**    | Window size is constant                     | e.g., "Find max sum of subarray size k"                     |
| **Variable Window** | Window expands/contracts based on condition | e.g., "Find longest substring without repeating characters" |

---

### 🧮 1. Fixed Sliding Window — Maximum Sum Subarray of Size `k`

#### 🔹 Problem

Given an array of integers and a number `k`, find the maximum sum of any contiguous subarray of size `k`.

---

#### 🔹 Logic

1. Compute sum of first `k` elements.
2. Slide the window one step at a time:

   * Subtract the element leaving the window.
   * Add the new element entering the window.
3. Track the maximum sum.

---

#### 🔹 Python Code

```python
def max_sum_subarray(nums, k):
    if len(nums) < k:
        return None

    window_sum = sum(nums[:k])
    max_sum = window_sum

    for i in range(k, len(nums)):
        window_sum += nums[i] - nums[i - k]
        max_sum = max(max_sum, window_sum)

    return max_sum

## Example
nums = [2, 1, 5, 1, 3, 2]
k = 3
print("Max sum of subarray size", k, ":", max_sum_subarray(nums, k))
```

---

#### 🔹 Step-by-Step Example

Array: `[2, 1, 5, 1, 3, 2]`, `k = 3`

| Window | Elements | Sum | Max |
| ------ | -------- | --- | --- |
| [0:2]  | [2,1,5]  | 8   | 8   |
| [1:3]  | [1,5,1]  | 7   | 8   |
| [2:4]  | [5,1,3]  | 9   | 9   |
| [3:5]  | [1,3,2]  | 6   | 9   |

✅ **Result: 9**

---

### 🧠 2. Variable Sliding Window — Longest Substring Without Repeating Characters

#### 🔹 Problem

Given a string `s`, find the length of the **longest substring** that has **no repeating characters**.

---

#### 🔹 Logic

Use two pointers `left` and `right` to represent the window.

* Expand `right` by adding characters.
* If duplicate found → shrink `left` until duplicate removed.
* Track max window length.

---

#### 🔹 Python Code

```python
def longest_unique_substring(s):
    left = 0
    seen = {}
    max_len = 0

    for right, ch in enumerate(s):
        if ch in seen and seen[ch] >= left:
            left = seen[ch] + 1  ## move left past the duplicate
        seen[ch] = right
        max_len = max(max_len, right - left + 1)

    return max_len

## Example
s = "abcabcbb"
print("Longest unique substring length:", longest_unique_substring(s))
```

---

#### 🔹 Step-by-Step Example

`s = "abcabcbb"`

| Step | Window | Action                         | MaxLen |
| ---- | ------ | ------------------------------ | ------ |
| 0    | `a`    | Add 'a'                        | 1      |
| 1    | `ab`   | Add 'b'                        | 2      |
| 2    | `abc`  | Add 'c'                        | 3      |
| 3    | `abca` | Duplicate 'a' → move left to 1 | 3      |
| 4    | `bcab` | Duplicate 'b' → move left to 2 | 3      |
| 5    | `cabc` | Duplicate 'c' → move left to 3 | 3      |
| 6    | `abcb` | Duplicate 'b' → move left to 5 | 3      |
| 7    | `b`    | End                            | 3      |

✅ **Result:** `3` → substring `"abc"`

---

### 🧩 3. Smallest Subarray with Sum ≥ Target

#### 🔹 Problem

Given an array of positive numbers and a positive number `S`, find the **length of the smallest contiguous subarray** whose sum ≥ `S`.

---

#### 🔹 Logic

* Expand window by moving `right`.
* When `window_sum ≥ S`, shrink `left` to minimize window.
* Track smallest length.

---

#### 🔹 Python Code

```python
import math

def smallest_subarray_sum(S, nums):
    left = 0
    window_sum = 0
    min_len = math.inf

    for right in range(len(nums)):
        window_sum += nums[right]
        while window_sum >= S:
            min_len = min(min_len, right - left + 1)
            window_sum -= nums[left]
            left += 1
    return 0 if min_len == math.inf else min_len

## Example
nums = [2, 1, 5, 2, 3, 2]
S = 7
print("Smallest subarray length:", smallest_subarray_sum(S, nums))
```

---

#### 🔹 Step-by-Step Example

`nums = [2, 1, 5, 2, 3, 2]`, `S = 7`

| Step | Window  | Sum | Shrink?                   | MinLen |
| ---- | ------- | --- | ------------------------- | ------ |
| 0    | [2]     | 2   | No                        | ∞      |
| 1    | [2,1]   | 3   | No                        | ∞      |
| 2    | [2,1,5] | 8   | Yes → shrink left → [1,5] | 3      |
| 3    | [1,5,2] | 8   | shrink → [5,2]            | 2      |
| 4    | [5,2,3] | 10  | shrink → [2,3] (sum <7)   | 2      |

✅ **Result:** `2` (subarray `[5,2]`)

---

### ⚙️ 4. Longest Subarray with Sum ≤ Target

#### 🔹 Problem

Find the longest subarray whose sum is less than or equal to `K`.

---

#### 🔹 Logic

1. Keep expanding `right` and accumulate sum.
2. While `window_sum > K`, shrink from left.
3. Track longest length.

---

#### 🔹 Python Code

```python
def longest_subarray_sum_leq_k(nums, k):
    left = 0
    window_sum = 0
    max_len = 0

    for right in range(len(nums)):
        window_sum += nums[right]
        while window_sum > k and left <= right:
            window_sum -= nums[left]
            left += 1
        max_len = max(max_len, right - left + 1)

    return max_len

## Example
nums = [1, 2, 1, 0, 1, 1, 0]
k = 4
print("Longest subarray sum <= K:", longest_subarray_sum_leq_k(nums, k))
```

---

#### 🔹 Step-by-Step Example

`nums = [1,2,1,0,1,1,0]`, `K = 4`

| Step | Window      | Sum | Action             | MaxLen |
| ---- | ----------- | --- | ------------------ | ------ |
| 0    | [1]         | 1   | ok                 | 1      |
| 1    | [1,2]       | 3   | ok                 | 2      |
| 2    | [1,2,1]     | 4   | ok                 | 3      |
| 3    | [1,2,1,0]   | 4   | ok                 | 4      |
| 4    | [1,2,1,0,1] | 5   | shrink → [2,1,0,1] | 4      |
| 5    | [2,1,0,1,1] | 5   | shrink → [1,0,1,1] | 4      |
| 6    | [0,1,1,0]   | 2   | ok                 | 4      |

✅ **Result:** `4`

---


Perfect — this is a **Sliding Window** + **Prefix-Suffix pattern** problem, very popular in interviews (sometimes called “**Maximum Points from Cards**”).

Let’s go step-by-step with full explanation 👇

---

### 🧩 Problem Statement

You are given an array `cards`, where each element represents the **points on a card**.
You can take exactly **k cards**, but **only from either end** (left or right).

Find the **maximum total score** you can achieve.

---

### **Example Input**

```python
cards = [2, 11, 4, 5, 3, 9, 2]
k = 3
```

### **Expected Output**

```
24
```

---

### 🧠 Logic

### Key Idea:

Instead of trying all combinations of picking cards from the start or end,
we can think **in reverse**:

➡️ You are leaving **`n - k` cards** in the middle.
If you minimize the sum of these **remaining (n - k)** cards,
you maximize your picked score.

---

### Steps

1. Compute total sum of all cards.
2. Use a **sliding window** of size `n - k` to find the **minimum subarray sum**.
3. Subtract this minimum sum from total sum → gives **maximum score**.

---

### Example Intuition

For `cards = [2,11,4,5,3,9,2]`, `k = 3`

* Total sum = 36
* You must leave `n - k = 4` cards.
* Find the **minimum sum subarray of length 4**.
* Subtract that from total → `36 - min_sub_sum = max_score`.

---

### 💻 Python Code

```python
def maxScore(cards, k):
    n = len(cards)
    total = sum(cards)

    if k == n:
        return total

    window_size = n - k
    window_sum = sum(cards[:window_size])
    min_sub_sum = window_sum

    # Slide the window
    for i in range(window_size, n):
        window_sum += cards[i] - cards[i - window_size]
        min_sub_sum = min(min_sub_sum, window_sum)

    return total - min_sub_sum


# Example Input
cards = [2, 11, 4, 5, 3, 9, 2]
k = 3

# Output
result = maxScore(cards, k)
print("Maximum score:", result)
```

---

### 🧾 Step-by-Step Trace Table

| Step | Window (size 4) | Window Sum | min_sub_sum | total - min_sub_sum | max_score |
| ---- | --------------- | ---------- | ----------- | ------------------- | --------- |
| Init | [2,11,4,5]      | 22         | 22          | 36 - 22 = 14        | 14        |
| 1    | [11,4,5,3]      | 23         | 22          | 13                  | 14        |
| 2    | [4,5,3,9]       | 21         | 21          | 15                  | 15        |
| 3    | [5,3,9,2]       | 19         | **19**      | **17**              | **17**    |

✅ Minimum middle sum = 19
✅ Maximum score = 36 - 19 = **17**

---

Wait... that’s **not 24**, right?
Let’s check if we took the wrong subarray size.

👉 We want **k = 3**, so we can also directly simulate prefix + suffix pick.

---

### Alternate (More Intuitive) Approach — Prefix + Suffix Sum

Compute:

* Take first `i` cards from start,
* Take remaining `(k - i)` cards from end,
* Maximize total.

---

#### 💻 Code (Prefix + Suffix Method)

```python
def maxScore(cards, k):
    n = len(cards)
    total = 0

    # Take first k cards initially
    for i in range(k):
        total += cards[i]

    max_score = total

    # Start swapping cards from start with end one by one
    for i in range(1, k + 1):
        total = total - cards[k - i] + cards[-i]
        max_score = max(max_score, total)

    return max_score


# Example
cards = [2, 11, 4, 5, 3, 9, 2]
k = 3
print("Maximum score:", maxScore(cards, k))
```

---

### 🧾 Step-by-Step Trace (Prefix + Suffix)

| Step | Take from Start | Take from End | Current Selection | Score  | Max Score |
| ---- | --------------- | ------------- | ----------------- | ------ | --------- |
| 1    | [2, 11, 4]      | []            | [2,11,4]          | 17     | 17        |
| 2    | [2,11]          | [2]           | [2,11,2]          | 15     | 17        |
| 3    | [2]             | [9,2]         | [2,9,2]           | 13     | 17        |
| 4    | []              | [5,9,2]       | [5,9,2]           | **16** | 17        |

✅ **Maximum Score = 17**

---

### 🎯 Final Output

```
Maximum score: 17
```

---

**“Maximum Sum of Distinct Subarrays With Length K”** (LeetCode #2461).


---

### 🧩 Problem

We are given:

* `nums`: an integer array
* `k`: subarray length

We need:

* The **maximum possible sum** of a **subarray of length `k`**
* The subarray must have **all unique elements**

---

### 🧠 Approach — Sliding Window + Hash Set

We can use:

* A **sliding window** of size up to `k`
* A **set** to ensure all elements are unique
* A **running sum** to track the current window sum

We slide the window across the array:

1. Expand the window by adding the current element.
2. If the element is already in the set → shrink the window from the left until it becomes unique again.
3. If the window size equals `k`:

   * Update `max_sum`
   * Remove the leftmost element to slide the window forward.

---

### 💻 Python Code

```python
def maxSumDistinctSubarray(nums, k):
    seen = set()
    left = 0
    current_sum = 0
    max_sum = 0

    for right in range(len(nums)):
        # If duplicate found, shrink window until unique
        while nums[right] in seen:
            seen.remove(nums[left])
            current_sum -= nums[left]
            left += 1

        # Add current element
        seen.add(nums[right])
        current_sum += nums[right]

        # If window size exceeds k, shrink from left
        if right - left + 1 > k:
            seen.remove(nums[left])
            current_sum -= nums[left]
            left += 1

        # If valid window of size k → update max
        if right - left + 1 == k:
            max_sum = max(max_sum, current_sum)

    return max_sum


# Example
nums = [3, 2, 2, 3, 4, 6, 7, 7, -1]
k = 4
print("Maximum sum of unique subarray:", maxSumDistinctSubarray(nums, k))
```

---

### ✅ Output

```
Maximum sum of unique subarray: 20
```

---

### 🧾 Step-by-Step Trace

| Step | Window Elements | Unique? | Current Sum | Valid (size=k)? | Max Sum |
| ---- | --------------- | ------- | ----------- | --------------- | ------- |
| 1    | [3,2,2,3]       | ❌       | -           | -               | 0       |
| 2    | [2,3,4,6]       | ✅       | 15          | ✅               | 15      |
| 3    | [3,4,6,7]       | ✅       | **20**      | ✅               | **20**  |
| 4    | [4,6,7,7]       | ❌       | -           | -               | 20      |
| 5    | [6,7,7,-1]      | ❌       | -           | -               | 20      |

✅ The maximum unique-sum subarray = `[3,4,6,7]`
✅ Sum = **20**

---

### ⚙️ Complexity

| Metric    | Complexity                                        |
| --------- | ------------------------------------------------- |
| **Time**  | O(n) — each element is added/removed at most once |
| **Space** | O(k) — for the set storing up to `k` elements     |

---

Would you like me to show a **visual sliding window animation (text/mermaid diagram)** for this example to clearly illustrate how duplicates are handled and the window moves?


---
### ⚙️ Complexity

| Metric    | Complexity                  |
| --------- | --------------------------- |
| **Time**  | O(k) — single pass          |
| **Space** | O(1) — constant extra space |

---

✅ **Summary:**

* **Sliding Window Version:** Intuitive if you think in terms of what’s *left out*.
* **Prefix+Suffix Version:** Intuitive if you think in terms of *what’s taken*.

---

Would you like me to show both approaches (window vs prefix-suffix) side by side with **diagrammatic explanation**?


---
### 🎯 Quick Recap

| Category  | Problem                           | Window Type | Output Example |
| --------- | --------------------------------- | ----------- | -------------- |
| Subarray  | Max sum subarray of size k        | Fixed       | 9              |
| Substring | Longest substring without repeats | Variable    | 3              |
| Subarray  | Smallest subarray with sum ≥ S    | Variable    | 2              |
| Subarray  | Longest subarray with sum ≤ K     | Variable    | 4              |

---

### 🧠 Pro Tip: Pattern Summary

| Pattern               | Left Pointer Movement            | Used For                             |
| --------------------- | -------------------------------- | ------------------------------------ |
| Fixed window          | Always moves after k elements    | Fixed-length problems                |
| Expand–contract       | Moves only when condition breaks | Longest/Shortest range problems      |
| Map/Set inside window | To track characters/elements     | Distinct or frequency-based problems |

---

Would you like me to continue this series with **sliding window problems that involve characters frequency** (like “minimum window substring” or “anagrams in string”)?
Those are the **next-level variable window** challenges commonly asked in FAANG interviews.


## Step 3  -> Merging Interval

Great — I’ll cover **four core interval problems**. For each one I’ll give:

👉 **Problem → Logic → Step-by-step trace → Python code → Sample input & output**

The four problems:

1. **Merge Intervals**
2. **Minimum Meeting Rooms (Meeting Rooms II)**
3. **Insert Interval**
4. **Maximum Number of Non-overlapping Intervals (Interval Scheduling / Select intervals)**

---

### 1) Merge Intervals

###  Problem

Given a list of intervals `[[s,e], ...]`, merge all overlapping intervals and return the merged list.

###  Logic

1. Sort intervals by start time.
2. Iterate and keep a `merged` list.
3. If current interval overlaps with last merged (`curr_start <= last_end`), update last end = `max(last_end, curr_end)`.
4. Else append current interval.

###  Step-by-step trace (example)

Input: `[[1,3],[2,6],[8,10],[15,18]]`
Sorted (already): `[[1,3],[2,6],[8,10],[15,18]]`

* merged = []
* take [1,3] → merged = [[1,3]]
* take [2,6] → overlaps with [1,3] → merge → [1,6] → merged = [[1,6]]
* take [8,10] → no overlap → append → merged = [[1,6],[8,10]]
* take [15,18] → append → merged = [[1,6],[8,10],[15,18]]

###  Python code

```python
def merge_intervals(intervals):
    if not intervals:
        return []
    intervals.sort(key=lambda x: x[0])
    merged = []
    for interval in intervals:
        if not merged or merged[-1][1] < interval[0]:
            merged.append(interval.copy())
        else:
            merged[-1][1] = max(merged[-1][1], interval[1])
    return merged

# Sample run
intervals = [[1,3],[2,6],[8,10],[15,18]]
print("Input:", intervals)
print("Merged:", merge_intervals(intervals))
```

###  Sample input & output

Input: `[[1,3],[2,6],[8,10],[15,18]]`
Output: `[[1,6],[8,10],[15,18]]`

---

### 2) Minimum Meeting Rooms (Meeting Rooms II)

###  Problem

Given an array of meeting time intervals `[[s,e], ...]`, find the **minimum number of conference rooms** required to schedule all meetings (no overlaps in same room).

###  Logic (two common approaches)

**Approach A — Min-heap (priority queue)**:

1. Sort intervals by start time.
2. Use a min-heap that stores end times of meetings currently occupying rooms.
3. For each interval:

   * If heap not empty and smallest end ≤ current start → reuse that room (pop heap).
   * Push current end into heap.
4. Heap size at end = number of rooms needed.

**Approach B — Sweep-line (two arrays)**:

1. Separate starts and ends, sort both.
2. Iterate through starts; maintain `rooms` count:

   * If start < earliest end → need a new room.
   * Else start >= earliest end → reuse (move end pointer).
3. Track max rooms used.

I'll implement the heap approach.

###  Step-by-step trace (example)

Input: `[[0,30],[5,10],[15,20]]` sorted: same

* heap = []
* [0,30]: heap empty → push 30 → heap = [30]
* [5,10]: earliest end 30 > 5 → need new room → push 10 → heap = [10,30]
* [15,20]: earliest end 10 <= 15 → pop 10 (reuse room), then push 20 → heap = [20,30]
  End → heap size = 2 → need 2 rooms.

###  Python code

```python
import heapq

def min_meeting_rooms(intervals):
    if not intervals:
        return 0
    intervals.sort(key=lambda x: x[0])
    heap = []  # min-heap of end times
    for start, end in intervals:
        if heap and heap[0] <= start:
            heapq.heappop(heap)  # room becomes free, reuse
        heapq.heappush(heap, end)
    return len(heap)

# Sample run
intervals = [[0,30],[5,10],[15,20]]
print("Intervals:", intervals)
print("Min rooms required:", min_meeting_rooms(intervals))
```

###  Sample input & output

Input: `[[0,30],[5,10],[15,20]]`
Output: `2`

---

### 3) Insert Interval

###  Problem

Given a list of non-overlapping intervals sorted by start time and a new interval `newInterval`, insert it into intervals and merge if necessary. Return the updated list.

###  Logic

1. Add all intervals that end before `newInterval` starts → they come unchanged.
2. Merge all intervals that overlap with `newInterval` by updating `newInterval = [min(start, new_start), max(end, new_end)]`.
3. Add the merged `newInterval`.
4. Add remaining intervals that start after `newInterval` ends.

###  Step-by-step trace (example)

Input: `intervals = [[1,2],[3,5],[6,7],[8,10],[12,16]]`, `newInterval = [4,8]`

* Add intervals ending before 4: [1,2], [3,5]? [3,5] overlaps because 5 >= 4. So only [1,2] goes to result.
* Merge overlaps:

  * start merge with [3,5] → new = [min(4,3)=3, max(8,5)=8] => [3,8]
  * next [6,7] overlaps → new becomes [3,8]
  * next [8,10] overlaps (8 <= 10) → new becomes [3, max(8,10)=10] => [3,10]
  * next [12,16] does not overlap (12 > 10)
* result = [[1,2], [3,10], [12,16]]

###  Python code

```python
def insert_interval(intervals, new_interval):
    res = []
    i = 0
    n = len(intervals)
    new_start, new_end = new_interval

    # add intervals before new_interval
    while i < n and intervals[i][1] < new_start:
        res.append(intervals[i])
        i += 1

    # merge overlapping
    while i < n and intervals[i][0] <= new_end:
        new_start = min(new_start, intervals[i][0])
        new_end = max(new_end, intervals[i][1])
        i += 1
    res.append([new_start, new_end])

    # add remaining intervals
    while i < n:
        res.append(intervals[i])
        i += 1

    return res

# Sample run
intervals = [[1,2],[3,5],[6,7],[8,10],[12,16]]
new_interval = [4,8]
print("Input intervals:", intervals)
print("New interval:", new_interval)
print("After insert:", insert_interval(intervals, new_interval))
```

###  Sample input & output

Input: `intervals = [[1,2],[3,5],[6,7],[8,10],[12,16]]`, `newInterval=[4,8]`
Output: `[[1,2],[3,10],[12,16]]`

---

### 4) Maximum Number of Non-overlapping Intervals (Interval Scheduling)

###  Problem

Given a list of intervals `[[s,e], ...]`, find the **maximum number of non-overlapping intervals** you can select (i.e., schedule the maximum number of non-overlapping tasks).

This is the classic **interval scheduling maximization** problem.

###  Logic (Greedy)

1. Sort intervals by **end time** ascending.
2. Iterate and greedily pick an interval if its start ≥ end time of last picked interval.
3. This maximizes the count.

###  Step-by-step trace (example)

Input: `[[1,3],[2,4],[3,5],[0,6],[5,7],[8,9]]`
Sorted by end: `[[1,3],[2,4],[3,5],[0,6],[5,7],[8,9]]` (already mostly sorted)

* pick [1,3], last_end=3, count=1
* [2,4]: start 2 < last_end 3 → skip
* [3,5]: start 3 >= 3 → pick, last_end=5, count=2
* [0,6]: start 0 <5 → skip
* [5,7]: start 5 >=5 → pick, last_end=7, count=3
* [8,9]: start 8 >=7 → pick, last_end=9, count=4

Max non-overlapping = 4

###  Python code

```python
def max_non_overlapping_intervals(intervals):
    if not intervals:
        return 0
    intervals.sort(key=lambda x: x[1])  # sort by end time
    count = 0
    last_end = -10**18
    for s, e in intervals:
        if s >= last_end:
            count += 1
            last_end = e
    return count

# Sample run
intervals = [[1,3],[2,4],[3,5],[0,6],[5,7],[8,9]]
print("Intervals:", intervals)
print("Max non-overlapping intervals:", max_non_overlapping_intervals(intervals))
```

###  Sample input & output

Input: `[[1,3],[2,4],[3,5],[0,6],[5,7],[8,9]]`
Output: `4` (e.g., pick `[[1,3],[3,5],[5,7],[8,9]]`)

---

#### Complexity summary

| Problem                               | Time Complexity                                      | Space Complexity |
| ------------------------------------- | ---------------------------------------------------- | ---------------- |
| Merge Intervals                       | O(n log n) (sort) + O(n)                             | O(n) output      |
| Min Meeting Rooms (heap)              | O(n log n) (sort) + O(n log n) heap ops              | O(n) heap        |
| Insert Interval                       | O(n) if already sorted input (or O(n log n) to sort) | O(n)             |
| Interval Scheduling (max non-overlap) | O(n log n) (sort) + O(n)                             | O(1) extra       |

---

If you want:

* runnable script combining all four functions with interactive input, or
* visual timeline ASCII traces for each example, or
* solutions in another language (Node/Java/JS) — I can produce them next. Which would you like?


## Step 4 --> Two Pointers pattern

Excellent — the **Two Pointers pattern** is one of the most fundamental and versatile DSA techniques 💡

Let’s go through **4 representative problems** that cover its major use cases:

| ###   | Category         | Example Problem                                                       |
| --- | ---------------- | --------------------------------------------------------------------- |
| 1️⃣ | Pair Sums        | Two Sum II – Find if pair exists with given target (sorted array)     |
| 2️⃣ | Sorting / Merge  | Merge Two Sorted Arrays                                               |
| 3️⃣ | Palindrome Check | Check if a string is palindrome (and after removing non-alphanumeric) |
| 4️⃣ | Move Elements    | Move Zeroes or Partition Array                                        |

---

We’ll follow your requested structure:

👉 **Problem → Logic → Step-by-Step Trace → Python Code → Sample Input & Output**

---

#### 1️⃣ Two Pointers — Pair Sum (Sorted Array)

##### 🧩 Problem

Given a **sorted array** and a target, find if there exist two numbers that sum up to the target.

**Example:**
`arr = [1, 2, 3, 4, 6], target = 6`
Output → `[1, 3]` (because 2 + 4 = 6)

---

##### ⚙️ Logic

1. Initialize `left = 0` and `right = len(arr)-1`.
2. Compute `curr_sum = arr[left] + arr[right]`.
3. If `curr_sum == target` → found pair.
   If `curr_sum < target` → increase `left` (need bigger sum).
   If `curr_sum > target` → decrease `right` (need smaller sum).
4. Continue until `left >= right`.

---

##### 🔍 Step-by-step Trace

| left | right | arr[left] | arr[right] | sum | Action                |
| ---- | ----- | --------- | ---------- | --- | --------------------- |
| 0    | 4     | 1         | 6          | 7   | > target → move right |
| 0    | 3     | 1         | 4          | 5   | < target → move left  |
| 1    | 3     | 2         | 4          | 6   | = target → found      |

✅ Pair = (2, 4)

---

##### 💻 Python Code

```python
def two_sum_sorted(arr, target):
    left, right = 0, len(arr) - 1
    while left < right:
        s = arr[left] + arr[right]
        if s == target:
            return [left, right]
        elif s < target:
            left += 1
        else:
            right -= 1
    return [-1, -1]

### Sample run
arr = [1, 2, 3, 4, 6]
target = 6
print("Input:", arr, "Target:", target)
print("Pair indices:", two_sum_sorted(arr, target))
```

##### 🎯 Output

```
Input: [1, 2, 3, 4, 6] Target: 6
Pair indices: [1, 3]
```

---

#### 2️⃣ Two Pointers — Merging Sorted Arrays

##### 🧩 Problem

Merge two sorted arrays into one sorted array (like merge step in Merge Sort).

**Example:**
`a = [1, 3, 5]`, `b = [2, 4, 6]`
Output → `[1, 2, 3, 4, 5, 6]`

---

##### ⚙️ Logic

1. Use two pointers `i`, `j` starting from start of each list.
2. Compare elements, append the smaller one to `merged`.
3. Move that pointer forward.
4. When one list ends, append remaining elements of the other.

---

##### 🔍 Step-by-step Trace

| i   | j | a[i] | b[j] | merged        | Action      |
| --- | - | ---- | ---- | ------------- | ----------- |
| 0   | 0 | 1    | 2    | [1]           | move i      |
| 1   | 0 | 3    | 2    | [1,2]         | move j      |
| 1   | 1 | 3    | 4    | [1,2,3]       | move i      |
| 2   | 1 | 5    | 4    | [1,2,3,4]     | move j      |
| 2   | 2 | 5    | 6    | [1,2,3,4,5]   | move i      |
| end | 2 | -    | 6    | [1,2,3,4,5,6] | append rest |

---

##### 💻 Python Code

```python
def merge_sorted_arrays(a, b):
    i, j = 0, 0
    merged = []
    while i < len(a) and j < len(b):
        if a[i] < b[j]:
            merged.append(a[i])
            i += 1
        else:
            merged.append(b[j])
            j += 1
    ### Append remaining
    merged.extend(a[i:])
    merged.extend(b[j:])
    return merged

### Sample run
a = [1,3,5]
b = [2,4,6]
print("Merged:", merge_sorted_arrays(a,b))
```

##### 🎯 Output

```
Merged: [1, 2, 3, 4, 5, 6]
```

---

#### 3️⃣ Two Pointers — Palindrome Check (String)

##### 🧩 Problem

Check if a given string is a palindrome **ignoring case and non-alphanumeric** characters.

**Example:**
`"A man, a plan, a canal: Panama"` → True
`"race a car"` → False

---

##### ⚙️ Logic

1. Clean string → keep only alphanumeric, make lowercase.
2. Use two pointers `left` and `right`.
3. While `left < right`:

   * If characters differ → return False.
   * Else move inward (`left += 1, right -= 1`).
4. Return True.

---

##### 🔍 Step-by-step Trace

`s = "A man, a plan, a canal: Panama"`
clean → `"amanaplanacanalpanama"`
Check pairs:
a==a ✅, m==m ✅, a==a ✅, ... all match → ✅ Palindrome

---

##### 💻 Python Code

```python
def is_palindrome(s):
    cleaned = ''.join(ch.lower() for ch in s if ch.isalnum())
    left, right = 0, len(cleaned) - 1
    while left < right:
        if cleaned[left] != cleaned[right]:
            return False
        left += 1
        right -= 1
    return True

### Sample run
print(is_palindrome("A man, a plan, a canal: Panama"))  ### True
print(is_palindrome("race a car"))  ### False
```

##### 🎯 Output

```
True
False
```

---

#### 4️⃣ Two Pointers — Move Zeroes / Partition Array

##### 🧩 Problem

Move all zeroes in array to the end **without changing the order of non-zero elements**.

**Example:**
`[0,1,0,3,12]` → `[1,3,12,0,0]`

---

##### ⚙️ Logic

1. Use two pointers:

   * `slow`: position to place next non-zero.
   * `fast`: iterate over array.
2. If `arr[fast] != 0`, swap `arr[slow]` and `arr[fast]` then increment `slow`.
3. Continue till end.

---

##### 🔍 Step-by-step Trace

| fast | arr          | slow | Action                                     |
| ---- | ------------ | ---- | ------------------------------------------ |
| 0    | [0,1,0,3,12] | 0    | 0=0 → skip                                 |
| 1    | [0,1,0,3,12] | 0    | swap arr[0], arr[1] → [1,0,0,3,12], slow=1 |
| 2    | [1,0,0,3,12] | 1    | 0=0 → skip                                 |
| 3    | [1,0,0,3,12] | 1    | swap arr[1], arr[3] → [1,3,0,0,12], slow=2 |
| 4    | [1,3,0,0,12] | 2    | swap arr[2], arr[4] → [1,3,12,0,0]         |

✅ Done

---

##### 💻 Python Code

```python
def move_zeroes(nums):
    slow = 0
    for fast in range(len(nums)):
        if nums[fast] != 0:
            nums[slow], nums[fast] = nums[fast], nums[slow]
            slow += 1
    return nums

### Sample run
arr = [0,1,0,3,12]
print("Input:", arr)
print("Output:", move_zeroes(arr))
```

##### 🎯 Output

```
Input: [0, 1, 0, 3, 12]
Output: [1, 3, 12, 0, 0]
```

---

### ⚙️ Complexity Summary

| Problem             | Time   | Space  | Category          |
| ------------------- | ------ | ------ | ----------------- |
| Two Sum (Sorted)    | O(n)   | O(1)   | Pair sums         |
| Merge Sorted Arrays | O(n+m) | O(n+m) | Sorting / merging |
| Palindrome Check    | O(n)   | O(1)   | Palindrome        |
| Move Zeroes         | O(n)   | O(1)   | Partitioning      |

---

Would you like me to add **“Two Pointers Advanced”** next — e.g.,

* **3Sum**,
* **Container With Most Water**,
* **Trapping Rain Water**,
  which are real interview favorites using the same pattern?


🔥 Awesome — let’s now go **beyond basics** with **Advanced Two Pointers Patterns**,
the kind you’ll frequently see in **FAANG-level interviews**.

We’ll cover the **next 3 iconic problems**, each with your preferred format:

👉 **Problem → Logic → Step-by-step trace → Python code → Sample input & output**

---

### 1️⃣ 3Sum Problem

####  🧩 Problem

Given an integer array `nums`, return all **unique triplets** `[a,b,c]` such that
`a + b + c == 0`.

**Example:**
Input: `[-1, 0, 1, 2, -1, -4]`
Output: `[[-1, -1, 2], [-1, 0, 1]]`

---

####  ⚙️ Logic

1. Sort the array.
2. Fix one element (`i`), then use **two pointers** (`left` & `right`) to find the remaining pair.
3. Move pointers depending on sum:

   * If total < 0 → move `left` (increase sum).
   * If total > 0 → move `right` (decrease sum).
   * If total == 0 → add triplet & skip duplicates.
4. Continue until all i’s are processed.

---

####  🔍 Step-by-step Trace

For input `[-1, 0, 1, 2, -1, -4]`
Sort → `[-4, -1, -1, 0, 1, 2]`

| i   | fixed | left   | right | sum | Action            |
| --- | ----- | ------ | ----- | --- | ----------------- |
| 0   | -4    | 1 (-1) | 5 (2) | -3  | sum<0 → left++    |
| 0   | -4    | 2 (-1) | 5 (2) | -3  | left++            |
| ... | ...   | ...    | ...   | ... | ...               |
| 1   | -1    | 2 (-1) | 5 (2) | 0   | ✅ found [-1,-1,2] |
| 1   | -1    | 3 (0)  | 4 (1) | 0   | ✅ found [-1,0,1]  |

✅ Final = `[[-1,-1,2],[-1,0,1]]`

---

####  💻 Python Code

```python
def three_sum(nums):
    nums.sort()
    res = []
    for i in range(len(nums)):
        if i > 0 and nums[i] == nums[i-1]:
            continue
        left, right = i+1, len(nums)-1
        while left < right:
            s = nums[i] + nums[left] + nums[right]
            if s < 0:
                left += 1
            elif s > 0:
                right -= 1
            else:
                res.append([nums[i], nums[left], nums[right]])
                left += 1
                right -= 1
                while left < right and nums[left] == nums[left-1]:
                    left += 1
                while left < right and nums[right] == nums[right+1]:
                    right -= 1
    return res

### Sample run
nums = [-1, 0, 1, 2, -1, -4]
print("Input:", nums)
print("Triplets:", three_sum(nums))
```

####  🎯 Output

```
Input: [-1, 0, 1, 2, -1, -4]
Triplets: [[-1, -1, 2], [-1, 0, 1]]
```

---

### 2️⃣ Container With Most Water

####  🧩 Problem

Given `n` non-negative integers `height[i]`, each representing a vertical line on x-axis,
find **two lines that together hold the most water**.

**Example:**
Input: `[1,8,6,2,5,4,8,3,7]`
Output: `49`

---

####  ⚙️ Logic

1. Two pointers: `left = 0`, `right = n-1`.
2. Area = `(right - left) * min(height[left], height[right])`.
3. To maximize area:

   * Move the **shorter line’s pointer** inward (since taller one can still form larger area).
4. Keep track of max area.

---

####  🔍 Step-by-step Trace

| left | right | h[l] | h[r] | width | area | max_area | Action     |
| ---- | ----- | ---- | ---- | ----- | ---- | -------- | ---------- |
| 0    | 8     | 1    | 7    | 8     | 8    | 8        | move left  |
| 1    | 8     | 8    | 7    | 7     | 49   | 49       | move right |
| 1    | 7     | 8    | 3    | 6     | 18   | 49       | move right |
| ...  | ...   | ...  | ...  | ...   | ...  | ...      | ...        |

✅ Max area = **49**

---

####  💻 Python Code

```python
def max_area(height):
    left, right = 0, len(height) - 1
    max_water = 0
    while left < right:
        width = right - left
        curr_water = width * min(height[left], height[right])
        max_water = max(max_water, curr_water)
        if height[left] < height[right]:
            left += 1
        else:
            right -= 1
    return max_water

### Sample run
height = [1,8,6,2,5,4,8,3,7]
print("Input:", height)
print("Max Water Area:", max_area(height))
```

####  🎯 Output

```
Input: [1,8,6,2,5,4,8,3,7]
Max Water Area: 49
```

---

### 3️⃣ Trapping Rain Water

####  🧩 Problem

Given an elevation map, compute how much **water can be trapped** after raining.

**Example:**
Input: `[0,1,0,2,1,0,1,3,2,1,2,1]`
Output: `6`

---

####  ⚙️ Logic

Two-pointer approach avoids extra arrays:

1. Initialize `left = 0`, `right = n-1`, `left_max = 0`, `right_max = 0`.
2. While `left < right`:

   * If `height[left] < height[right]`:

     * If `height[left] >= left_max` → update `left_max`.
     * Else → water += `left_max - height[left]`.
     * Move `left += 1`
   * Else:

     * If `height[right] >= right_max` → update `right_max`.
     * Else → water += `right_max - height[right]`.
     * Move `right -= 1`
3. Return total trapped water.

---

####  🔍 Step-by-step Trace

`height = [0,1,0,2,1,0,1,3,2,1,2,1]`

| left | right | h[l] | h[r] | lmax | rmax | water | Action     |
| ---- | ----- | ---- | ---- | ---- | ---- | ----- | ---------- |
| 0    | 11    | 0    | 1    | 0    | 1    | 0     | move left  |
| 1    | 11    | 1    | 1    | 1    | 1    | 0     | move right |
| ...  | ...   | ...  | ...  | ...  | ...  | ...   | ...        |
| ...  | ...   | ...  | ...  | ...  | ...  | 6     | Done ✅     |

✅ Total trapped water = 6

---

####  💻 Python Code

```python
def trap_rain_water(height):
    left, right = 0, len(height) - 1
    left_max = right_max = 0
    water = 0
    while left < right:
        if height[left] < height[right]:
            if height[left] >= left_max:
                left_max = height[left]
            else:
                water += left_max - height[left]
            left += 1
        else:
            if height[right] >= right_max:
                right_max = height[right]
            else:
                water += right_max - height[right]
            right -= 1
    return water

### Sample run
height = [0,1,0,2,1,0,1,3,2,1,2,1]
print("Input:", height)
print("Trapped Water:", trap_rain_water(height))
```

####  🎯 Output

```
Input: [0,1,0,2,1,0,1,3,2,1,2,1]
Trapped Water: 6
```

---

### ⚙️ Complexity Summary

| Problem                   | Time  | Space | Category                  |
| ------------------------- | ----- | ----- | ------------------------- |
| 3Sum                      | O(n²) | O(1)  | Pair + Sorting            |
| Container With Most Water | O(n)  | O(1)  | Range-based               |
| Trapping Rain Water       | O(n)  | O(1)  | Range + Height Comparison |

---

✅ **Recap — Two Pointer Mastery Map**

| Level           | Concept                    | Typical Problems                     |
| --------------- | -------------------------- | ------------------------------------ |
| 🟢 Basic        | Opposite ends              | Two Sum, Palindrome, Move Zeroes     |
| 🟡 Intermediate | Sorted merging             | Merge Arrays, Insert Interval        |
| 🔵 Advanced     | Multi-pointer / Area logic | 3Sum, Container, Trapping Rain Water |

---

Would you like me to create the **next pattern after Two Pointers**, e.g.
👉 **Sliding Window Advanced** (longest substring without repeat, max sum subarray, etc.)
to continue your DSA pattern series?


Perfect — “**Trapping Rain Water**” is one of the **most classic Two Pointers problems** (sometimes also grouped under prefix/suffix patterns).
Let’s go step-by-step with **Problem → Logic → Step-by-Step Trace → Python Code → Input/Output** 💧

---

### 🧩 Problem: Trapping Rain Water

You are given an array `height[]` where each element represents the height of a bar in a histogram.
Rainwater can be trapped between the bars after it rains.

Your task: **Find how much total water is trapped.**

---

#### **Example Input**

```python
height = [4, 2, 0, 3, 2, 5]
```

#### **Expected Output**

```
9
```

---

### 🧠 Logic: Two Pointer Approach

Instead of calculating trapped water at each index using left and right arrays (O(n) space),
we can use **two pointers** — one from the left and one from the right — and track the **max heights** so far.

#### Steps:

1. Initialize `left = 0`, `right = len(height) - 1`
2. Maintain:

   * `left_max` → maximum height seen so far from the left
   * `right_max` → maximum height seen so far from the right
3. Move the smaller side inward:

   * If `height[left] < height[right]`:

     * If `height[left] < left_max`, water trapped = `left_max - height[left]`
     * Else update `left_max = height[left]`
     * Move `left` one step right
   * Else:

     * If `height[right] < right_max`, water trapped = `right_max - height[right]`
     * Else update `right_max = height[right]`
     * Move `right` one step left
4. Keep summing trapped water.

---

### 💻 Python Code

```python
def trap_rain_water(height):
    if not height:
        return 0

    left, right = 0, len(height) - 1
    left_max, right_max = height[left], height[right]
    water = 0

    while left < right:
        if height[left] < height[right]:
            left += 1
            if height[left] < left_max:
                water += left_max - height[left]
            else:
                left_max = height[left]
        else:
            right -= 1
            if height[right] < right_max:
                water += right_max - height[right]
            else:
                right_max = height[right]
    return water


# Input
height = [4, 2, 0, 3, 2, 5]

# Output
result = trap_rain_water(height)
print("Total trapped water:", result)
```

---

### 🧾 Step-by-Step Trace Table

| Step | left | right | height[left] | height[right] | left_max | right_max | Action                                   | Water Added             | Total Water |
| ---- | ---- | ----- | ------------ | ------------- | -------- | --------- | ---------------------------------------- | ----------------------- | ----------- |
| Init | 0    | 5     | 4            | 5             | 4        | 5         | Start                                    | 0                       | 0           |
| 1    | 1    | 5     | 2            | 5             | 4        | 5         | height[left] < height[right] → move left | left_max - h[l] = 4-2=2 | 2           |
| 2    | 2    | 5     | 0            | 5             | 4        | 5         | move left                                | 4-0=4                   | 6           |
| 3    | 3    | 5     | 3            | 5             | 4        | 5         | move left                                | 4-3=1                   | 7           |
| 4    | 4    | 5     | 2            | 5             | 4        | 5         | move left                                | 4-2=2                   | 9           |
| 5    | 5    | 5     | 5            | 5             | -        | -         | left == right (stop)                     | -                       | **9**       |

✅ **Total Water = 9**

---

### 🎯 Final Output

```
Total trapped water: 9
```

---

### ⚙️ Time & Space Complexity

| Metric    | Complexity                  |
| --------- | --------------------------- |
| **Time**  | O(n) → single traversal     |
| **Space** | O(1) → constant extra space |

---

### 💡 Quick Visualization

```
Bars:     [4, 2, 0, 3, 2, 5]
Water:      ░░░░░████░░
Total = 9 units
```

---

Would you like me to show the **Prefix/Suffix array approach** too (same problem, but using two auxiliary arrays for clarity comparison)?
